In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetSpec, DatasetBag, Workflow, ExecutionConfiguration, VersionPart
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
# RID of source dataset, if any.
source_dataset = '6-ACEJ' # Prod '4-YZPA' test'5-2GGR' Dev '5-YSK2' test'6-ACAJ'

EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
workflow_instance = EA.create_workflow(
    name="AI-Readi data template",
    workflow_type="Test Workflow",
)

# Configuration instance.
config = ExecutionConfiguration(
    datasets=[DatasetSpec(rid=source_dataset, version=EA.dataset_version(source_dataset), materialize=True)],
    # Materialize set to False if you only need the metadata from the bag, and not the assets,
    workflow=workflow_instance,
    description="Template instance of an AI-Readi data workflow")

# Initialize execution
execution = EA.create_execution(config)


In [ ]:
ds_bag = execution.datasets[0]

In [ ]:
sys_col = ['RMB', 'RCB', 'RMT', 'RCT']
subject = ds_bag.get_table_as_dataframe('Subject').drop(columns=sys_col).rename(columns={'RID': 'Subject'})
subject
observation = ds_bag.get_table_as_dataframe('Observation').drop(columns=sys_col).rename(columns={'RID': 'Observation',
                                                                                                'Date_of_Encoutner': 'Date_of_Encoutner_Observation'})
image = ds_bag.get_table_as_dataframe('Image').drop(columns=sys_col).rename(columns={'RID': 'Image',
                                                                                    'Date_of_Encoutner': 'Date_of_Encoutner_Image'})
device_detail = ds_bag.get_table_as_dataframe('Device').drop(columns=sys_col).rename(columns={'RID': 'Device'})
clinic = ds_bag.get_table_as_dataframe('AIREADI_Clinical_Records').drop(columns=sys_col).rename(columns={'RID': 'Clinic',
                                                                                                        'Date_of_Encounter': 'Date_of_Encoutner_Clinic'})

In [ ]:
clinic

In [ ]:
wide = subject.merge(observation, on='Subject', how='left')
wide = wide.merge(image, on='Observation', how='left')
wide = wide.merge(clinic, on='Observation', how='left')
# visit_occurance -> dateofencouter
# visit_id -> Observation ID
wide

In [ ]:

execution.upload_execution_outputs(clean_folder=True)